# Load Form 13 Nodes



## Imports

In [1]:
from dotenv import load_dotenv
import os

# Common data processing
import json
from pandas import DataFrame
import pandas as pd
from typing import List, Tuple, Union
from numpy.typing import ArrayLike

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

## Set up Neo4j and Langchain

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'


In [3]:
# Create a knowledge graph using Langchain's Neo4j integration.
# This will be used for direct querying of the knowledge graph. 
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)


## Prepare Neo4j indexes

In [4]:
# Create a uniqueness constraint on the managerCik property of Manager nodes 
kg.query('CREATE CONSTRAINT unique_manager IF NOT EXISTS FOR (n:Manager) REQUIRE n.managerCik IS UNIQUE')

# Create a uniqueness constraint on the managerCik property of Manager nodes 
kg.query('CREATE CONSTRAINT unique_company IF NOT EXISTS FOR (n:Company) REQUIRE n.cusip6 IS UNIQUE')


[]

## Load Form 13 sources

1. load the csv
2. for each line, merge two graph nodes, a `Manager` and a `Company`
3. for each `Manager` and `Company` pair, create an `OWNS_STOCK_IN` relationship between them


In [5]:
# Examine first row of csv
all_form13s = pd.read_csv('./data/form13.csv')

print(all_form13s.iloc[0].to_string()) 

source                     https://sec.gov/Archives/edgar/data/1049662/00...
managerCik                                                           1049662
managerName                                              GRIES FINANCIAL LLC
reportCalendarOrQuarter                                           2023-03-31
cusip6                                                                00206R
cusip                                                              00206R102
companyName                                                         AT&T INC
value                                                            359436000.0
shares                                                                 18672


In [6]:
%%time

# Load all form13s from the CSV file
all_form13s = pd.read_csv('./data/form13.csv')

# For each row in the CSV file, create a node for the manager and a node for the company
# Then connect the manager to the company with an OWNS_STOCK_IN relationship
for form13 in all_form13s.itertuples():
    print(form13.managerName, form13.companyName)
    kg.query(
        """
        MERGE (com:Company {cusip6: $cusip6})
            ON CREATE
                SET com.companyName = $companyName
                SET com.cusip = $cusip
        MERGE (man:Manager {managerCik: $managerCik})
            ON CREATE
                SET man.managerName = $managerName
        MERGE (man)-[:OWNS_STOCK_IN {value: $value, shares: $shares}]->(com)
        """, 
        params={
            'cusip6': form13.cusip6,
            'cusip': form13.cusip,
            'companyName': form13.companyName,
            'managerCik': form13.managerCik,
            'managerName': form13.managerName,
            'value': form13.value,
            'shares': form13.shares
        }
    )


GRIES FINANCIAL LLC AT&T INC
GRIES FINANCIAL LLC ABBOTT LABS
GRIES FINANCIAL LLC ABBVIE INC
GRIES FINANCIAL LLC ALTRIA GROUP INC
GRIES FINANCIAL LLC AMAZON COM INC
GRIES FINANCIAL LLC AMERICAN TOWER CORP NEW
GRIES FINANCIAL LLC AMGEN INC
GRIES FINANCIAL LLC APPLE INC
GRIES FINANCIAL LLC BANK NEW YORK MELLON CORP
GRIES FINANCIAL LLC BLACKROCK INC
GRIES FINANCIAL LLC BRISTOL-MYERS SQUIBB CO
GRIES FINANCIAL LLC BROADSTONE NET LEASE INC
GRIES FINANCIAL LLC BRUNSWICK CORP
GRIES FINANCIAL LLC THE CIGNA GROUP
GRIES FINANCIAL LLC CSX CORP
GRIES FINANCIAL LLC CVS HEALTH CORP
GRIES FINANCIAL LLC CHEVRON CORP NEW
GRIES FINANCIAL LLC CISCO SYS INC
GRIES FINANCIAL LLC CINTAS CORP
GRIES FINANCIAL LLC COCA COLA CO
GRIES FINANCIAL LLC CONAGRA BRANDS INC
GRIES FINANCIAL LLC CONOCOPHILLIPS
GRIES FINANCIAL LLC COPART INC
GRIES FINANCIAL LLC COSTCO WHSL CORP NEW
GRIES FINANCIAL LLC CUBESMART
GRIES FINANCIAL LLC DANAHER CORPORATION
GRIES FINANCIAL LLC DIAMONDROCK HOSPITALITY CO
GRIES FINANCIAL LLC DISNEY W

## Cypher queries to transform the Knowledge Graph

In [7]:
# Connect all `Company` nodes to their corresponding `Form` nodes
# based on the `cusip6` property

# MATCH a double node pattern, for the `Company` and `Form` nodes
# WHERE the `Company` and `Form` nodes have the same `cusip6` property
# MERGE to connect these pairs with a (:Company)-[:FILED]->(:Form) relationship
# RETURN a count of the number of relationships created or found (merged)
cypher = """
  MATCH (com:Company), (form:Form)
  WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
  RETURN count(*)
"""

kg.query(cypher)

[{'count(*)': 4}]

## Example Cypher queries

In [8]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:
Chunk {f10kItem: STRING, chunkSeqId: INTEGER, textEmbedding: LIST, chunkId: STRING, cik: STRING, cusip6: STRING, text: STRING, formId: STRING, source: STRING},Form {cusip6: STRING, source: STRING, formId: STRING, cik: STRING},Manager {managerCik: INTEGER, managerName: STRING},Company {cusip6: STRING, companyName: STRING, cusip: STRING}
Relationship properties are the following:
SECTION {f10kItem: STRING},OWNS_STOCK_IN {shares: INTEGER, value: FLOAT}
The relationships are the following:
(:Chunk)-[:PART_OF]->(:Form),(:Chunk)-[:NEXT]->(:Chunk),(:Form)-[:SECTION]->(:Chunk),(:Manager)-[:OWNS_STOCK_IN]->(:Company),(:Company)-[:FILED]->(:Form)


In [9]:
# Retrieve a company, working backwards from a specific Chunk...

# MATCH a 3 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form` which was `FILED` by a `Company`.
# RETURN the `companyName` property of the `Company` 
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(:Form)<-[:FILED]-(com:Company)
    RETURN com.companyName
    """

kg.query(cypher,
         params={'chunkIdParam': '0000950170-23-027948-item1-chunk0027'})

[{'com.companyName': 'NETAPP INC'}]

In [10]:
# Retrieve investors of a company, working backwards from a specific Chunk...

# MATCH a 4 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form` which was `FILED` by a `Company`
# which a `Manager` `OWNS_STOCK_IN`.
# RETURN a sentence about the manager's investment in the company 
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(:Form)<-[:FILED]-(com:Company)<-[investment:OWNS_STOCK_IN]-(mgr:Manager)
    
    RETURN mgr.managerName + " owns " + investment.shares + " of " + com.companyName + 
      " at a value of $" + apoc.number.format(investment.value) AS text
    """

kg.query(cypher,
         params={'chunkIdParam': '0000950170-23-027948-item1-chunk0027'})

[{'text': 'Narus Financial Partners, LLC owns 6386 of NETAPP INC at a value of $407,724,000'}]

## Prepare langchain for querying the Knowledge Graph

In [11]:
# OpenAI for creating embeddings
embeddings_model = OpenAIEmbeddings()

# Create a langchain vector store from the existing Neo4j knowledge graph.
vector_store = Neo4jVector.from_existing_graph(
    embedding=embeddings_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever
)

retrieval_query = """
MATCH (node)-[:PART_OF]->(:Form)<-[:FILED]-(com:Company)<-[investment:OWNS_STOCK_IN]-(mgr:Manager)
RETURN mgr.managerName + " owns " + investment.shares + " of " + com.companyName + 
    " at a value of $" + apoc.number.format(investment.value) + "." 
    + node.text AS text,
    score,
    { companyName: com.companyName, 
        managerName: mgr.managerName,
        shares: investment.shares,
        value: investment.value,
        source: node.source
      } as metadata
"""

vector_store_with_cypher = Neo4jVector.from_existing_index(
    embeddings_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=retrieval_query,
)

# Create a retriever from the vector store
retriever_with_cypher = vector_store_with_cypher.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain_with_cypher = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever_with_cypher
)

## Example user questions

In [14]:
question = 'In a single sentence, tell me about Netapp.'

In [18]:
question = 'In a single sentence, tell me about Netapp and their investors.'

In [19]:
# Vector search using the langchain retriever over the Neo4j vector store
retriever.get_relevant_documents(question)[0]

Document(page_content='\ntext: 31\n\n\n\n\n\xa0\n\n\nIt\nem 6.  \n[Reserved]\n\n\n \n\n\n\xa0\n\n\n32\n\n\n\n\n\xa0\n\n\nIt\nem 7.  \nManagement’s Discussion and Analysis of Financial Condition and Results of Operations\n\n\nThe following discussion of our financial condition and results of operations should be read together with the financial statements and the accompanying notes set forth under Item 8. – Financial Statements and Supplementary Data. The following discussion also contains trend information and other forward-looking statements that involve a number of risks and uncertainties. The Risk Factors set forth in Item 1A. – Risk Factors are hereby incorporated into the discussion by reference.\n\n\nExecutive Overview\n\n\nOur Company\n \n\n\nNetApp is a global cloud-led, data-centric software company that empowers customers with hybrid multicloud solutions built for a better future. Building on more than three decades of innovation, we give customers the freedom to manage appli

In [20]:
chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that empowers customers with hybrid multicloud solutions, and they work with a wide range of partners for their customers, including technology partners, value-added resellers, system integrators, OEMs, service providers, and distributors.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [21]:
chain_with_cypher(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that empowers customers with hybrid multicloud solutions, and their investors include Narus Financial Partners, LLC.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}